# Create album

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import *    
import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd

Using TensorFlow backend.


In [2]:
DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [6]:
import butterfly.album

In [7]:
# Choose your responses, predictors, and size for the image
predictor = 'serum_l'
response = 'plasma_l'
pixels = 40
features = 62

In [8]:
album = butterfly.album.create_album(DF, predictor, pixels)

In [10]:
import butterfly.CNN

In [11]:
#Decide which dataset you want to predict
response = [col for col in DF if col.startswith(response)]
response.append("patientID")
response.append("trimester")    
response_df = DF[response]
#Make sure your B trimesters are properly converted to 4

In [12]:
#Get every combination of patient IDs in pairs of m's
# Function which returns subset or r length from n 
from itertools import combinations 

m = 2
IDs = response_df['patientID'].unique()
IDs = IDs.tolist()
exclude = list(combinations(IDs, m))

In [13]:
#from joblib import Parallel, delayed
#import multiprocessing

#num_cores = multiprocessing.cpu_count()

#results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)

In [14]:
XX = album
yy = response_df.drop(['patientID', 'trimester'], axis =1 ).values

In [35]:
result = []

In [45]:
for cv in range(len(exclude)):
    #Create your calibration and validation datasets
    pt1ex = response_df.index[response_df['patientID'] == exclude[cv][0]].tolist()
    pt2ex = response_df.index[response_df['patientID'] == exclude[cv][1]].tolist()
    ptex = pt1ex+pt2ex

    #Divide in calibration and validation
    X_c = np.delete(XX, ptex, 0)
    X_c = X_c.reshape((X_c.shape[0], X_c.shape[1], pixels))
    y_c = np.delete(yy, ptex, 0)
    y_c = pd.DataFrame(StandardScaler().fit_transform(y_c))

    X_v = np.asarray([XX[i]  for i in ptex])
    X_v = X_v.reshape((X_v.shape[0], X_v.shape[1], pixels))
    y_v = np.asarray([yy[i] for i in ptex])
    y_v = pd.DataFrame(StandardScaler().fit_transform(y_v))
    
    #Create your CNN
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(pixels, pixels)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(features))
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X_c, y_c, epochs=300, verbose=0)
    
    # demonstrate prediction
    y_pred = model.predict(X_v, verbose = 0)
    y_pred = pd.DataFrame(y_pred)
    corr = y_v.corrwith(y_pred, axis = 0).values
    result.append(corr.clip(min=0))